# INSY 670 Group Project Part 2
Alice Liu, Diwei Zhu, Yingxin Jiang, Kexin Wang, Yichen Wang

## 1. Environment Setup

In [3]:
#!pip install snscrape
import snscrape.modules.twitter as sntwitter
import pandas as pd
import re

## 2. Twitter Data Scraping

In [3]:
key_word = "Zelda"  # Declare the key word used to search tweets
user_name = ""   # Declare a user name used to search tweets
from_date = "2020-01-01" # Declare a start date
end_date = '2021-03-19'  # Declare a end date
count = 5000             # The maximum number of tweets
tweets_list_keyword = [] # A list used to store the returned results for keyword search
tweets_list_user = []    # A list used to store the retuned results for user search

In [5]:
command_keyword = key_word+' since:'+from_date+' until:'+end_date # Define a string command for Scraper Api

print("Scraping data for keyword:", key_word)

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(command_keyword).get_items()):
    tweets_list_keyword.append([tweet.user.username, tweet.mentionedUsers, tweet.retweetedTweet]) # Append returned results to list
    if i>count:
        break;
        
# Create a dataframe from the tweets list above 
tweets_df_keyword = pd.DataFrame(tweets_list_keyword, columns=['username','mentionedusers', 'retweet'])
tweets_df_keyword.to_csv("tweets_keywords.csv",index=False) # Export to a csv file
print("Scraped data have been exported to the csv file")

Scraping data for keyword: Zelda
Scraped data have been exported to the csv file


## 3. Data  Preprocessing

In [4]:
# load the tweets data we scrape
tweets = pd.read_csv('tweets_keywords.csv')

# select columns we need to work with
tweets = tweets[['username','mentionedusers']]

# drop null values, aka tweets with no interactions between users
#tweets = tweets.dropna()

# reset index from 0
tweets = tweets.reset_index()

# drop the index column
df = tweets[['username','mentionedusers']]

In [5]:
df

,username,mentionedusers
0,RMDCade,NaN
1,RMDCade,NaN
2,Buster5ive,"[User(username='OnThisDayGaming', id=295461858..."
3,Skullivan_Bones,NaN
4,KdogGaming31,NaN
...,...,...
8130,miphacutiefish,NaN
8131,zais5,NaN
8132,Ninthusiast,NaN
8133,Waveon196,"[User(username='DizDotTV', id=1682684065, disp..."


In [6]:
import numpy as np
for i in range(len(df["username"])):
    string = "username="
    df["mentionedusers"]=df["mentionedusers"].replace(np.nan,string)

temp=[]
for i in range(len(df["username"])):
    temp_set = []
    temp_set.append(df["username"][i])
    temp_set.append(df["mentionedusers"][i])
    temp.append(temp_set)

for i in range(len(temp)):
    if temp[i][1]=="username=":
        temp_string ="username='"+temp[i][0]+"',"
        temp[i][1] = temp_string

<ipython-input-6-192ab271a59a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mentionedusers"]=df["mentionedusers"].replace(np.nan,string)


In [7]:
tweets = pd.DataFrame(temp,columns=["username","mentionedusers"])
tweets

,username,mentionedusers
0,RMDCade,"username='RMDCade',"
1,RMDCade,"username='RMDCade',"
2,Buster5ive,"[User(username='OnThisDayGaming', id=295461858..."
3,Skullivan_Bones,"username='Skullivan_Bones',"
4,KdogGaming31,"username='KdogGaming31',"
...,...,...
8130,miphacutiefish,"username='miphacutiefish',"
8131,zais5,"username='zais5',"
8132,Ninthusiast,"username='Ninthusiast',"
8133,Waveon196,"[User(username='DizDotTV', id=1682684065, disp..."


In [8]:
# calculate the number of mentioned users in a tweet
count = []
for i in range(tweets.shape[0]):
    value = tweets['mentionedusers'][i].count('username=')
    count.append(value)
tweets['#mentionedusers']=pd.Series(count)

In [9]:
tweets

,username,mentionedusers,#mentionedusers
0,RMDCade,"username='RMDCade',",1
1,RMDCade,"username='RMDCade',",1
2,Buster5ive,"[User(username='OnThisDayGaming', id=295461858...",1
3,Skullivan_Bones,"username='Skullivan_Bones',",1
4,KdogGaming31,"username='KdogGaming31',",1
...,...,...,...
8130,miphacutiefish,"username='miphacutiefish',",1
8131,zais5,"username='zais5',",1
8132,Ninthusiast,"username='Ninthusiast',",1
8133,Waveon196,"[User(username='DizDotTV', id=1682684065, disp...",1


In [10]:
# take a look at how many users are mentioned
tweets['#mentionedusers'].value_counts()

1     7224
2      453
3      108
16      81
15      62
4       41
11      35
5       25
7       25
8       23
10      16
9        9
6        9
17       7
12       5
25       4
14       2
13       2
48       2
19       2
Name: #mentionedusers, dtype: int64

In [11]:
# cleaning mentionedusers column
for i in range(tweets.shape[0]):
    string = tweets['mentionedusers'][i]
    users = re.findall(r'username=\S+', string)
    tweets['mentionedusers'][i]=users

# extract usernames
for i in range(tweets.shape[0]):
    for j in range(len(tweets['mentionedusers'][i])):
        newstring = tweets['mentionedusers'][i][j]
        newstring = newstring[10:]
        newstring = newstring[:-2]
        tweets['mentionedusers'][i][j] = newstring

<ipython-input-11-9ef898be9307>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets['mentionedusers'][i]=users


In [12]:
tweets

,username,mentionedusers,#mentionedusers
0,RMDCade,[RMDCade],1
1,RMDCade,[RMDCade],1
2,Buster5ive,[OnThisDayGaming],1
3,Skullivan_Bones,[Skullivan_Bones],1
4,KdogGaming31,[KdogGaming31],1
...,...,...,...
8130,miphacutiefish,[miphacutiefish],1
8131,zais5,[zais5],1
8132,Ninthusiast,[Ninthusiast],1
8133,Waveon196,[DizDotTV],1


## 4. Interaction records

In [13]:
user_mentioned = []
user_posting_tweet = []

In [14]:
for i in range(tweets.shape[0]):
    for j in range(len(tweets['mentionedusers'][i])):
        user_mentioned.append(tweets['mentionedusers'][i][j])
        user_posting_tweet.append(tweets['username'][i])

In [15]:
interactions = pd.DataFrame()
interactions['user_posting_tweet'] = user_posting_tweet
interactions['user_mentioned'] = user_mentioned

In [21]:
Type = []
for i in range(len(user_posting_tweet)):
    if user_posting_tweet[i]==user_mentioned[i]:
        Type.append("Tweet")
    else:
        Type.append("Non-tweet")
interactions["Type"]=Type

In [22]:
interactions.to_csv("users_interactions.csv",index=False) # Export to a csv file
interactions

,user_posting_tweet,user_mentioned,Type
0,RMDCade,RMDCade,Tweet
1,RMDCade,RMDCade,Tweet
2,Buster5ive,OnThisDayGaming,Non-tweet
3,Skullivan_Bones,Skullivan_Bones,Tweet
4,KdogGaming31,KdogGaming31,Tweet
...,...,...,...
12470,zais5,zais5,Tweet
12471,Ninthusiast,Ninthusiast,Tweet
12472,Waveon196,DizDotTV,Non-tweet
12473,tjamara_,Zelda_king13,Non-tweet


## 5. Mentioned Users Counts

In [32]:
# count how many times a user is mentioned by others
most = []
for i in range(tweets.shape[0]):
    for j in range(len(tweets['mentionedusers'][i])):
        user = tweets['mentionedusers'][i][j]
        most.append(user)

In [33]:
# see the ranking
most = pd.Series(most)
value_counts = most.value_counts()

# converting to df and assigning new names to the columns
df_value_counts = pd.DataFrame(value_counts)
df_value_counts = df_value_counts.reset_index()
df_value_counts.columns = ['username', 'mentioned_counts'] # change column names
df_value_counts.to_csv("mentioned_users_counts.csv",index=False) # Export to a csv file
df_value_counts

,username,mentioned_counts
0,Zeldathons,471
1,Zelda_king13,189
2,nerdist,128
3,Oh_Marie1,102
4,sithlord_zelda,102
...,...,...
4115,KinitaArts,1
4116,Correfoc_,1
4117,agusvikingoaria,1
4118,Manu_Aventuras,1


下面这一行code我没有run嘤嘤嘤嘤嘤朱老师加油加油加加油

In [ ]:
#### Scraping tweets a specific Twitter user’s account ####
command_user = 'from:'+user_name+' since:'+from_date+' until:'+end_date
print("Scraping data for user:",user_name)
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(command_user).get_items()): 
    tweets_list_user.append([tweet.date,tweet.id, tweet.content, tweet.user.username, tweet.url]) 
    if i > count:
        break;
# Create a dataframe from the tweets list above 
tweets_df_user = pd.DataFrame(tweets_list_user, columns=['Datetime','Tweet Id', 'Text', 'Username', 'url'])
tweets_df_user.to_csv("tweets_users1.csv",index=False) # Export to a csv file
print("Scraped data have been exported to the csv file")